In [1]:
'''
Constants, selection based upon https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm
Notation kept consistent with this article.
'''
WHITESPACE = ' '
MATCH = 3
MISMATCH = -3
W_1 = 2

In [2]:
def print_scoring_matrix(H: list[list[int]], seq1: str, seq2: str) -> None:
    row1 = WHITESPACE * 4
    for char in seq1:
        row1 += char + 3*WHITESPACE
    print(row1)
    
    for i, char in enumerate(seq2):
        row_string = char + WHITESPACE * 3
        for num in list(map(str, H[i])):
            row_string += num + ' ' * (4 - len(num))
        print(row_string)


def print_optimal_allignment(seq1_allign: str, seq2_allign: str) -> None:
    print(seq1_allign)
    row2 = ''
    for i, _ in enumerate(seq1_allign):
        row2 += '|' if seq1_allign[i] == seq2_allign[i] else WHITESPACE
    print(row2)
    print(seq2_allign)

In [3]:
#random sequence generation, for testing
import random


def random_sequence(sequence_length: int) -> str:
    return ''.join([random.choice('ATCG') for _ in range(sequence_length)])

In [4]:
#similarity score
def S(a: str, b: str) -> int:
    return MATCH if a == b else MISMATCH

In [5]:
def scoring_matrix(seq1: str, seq2: str) -> tuple:
    H = [[0]*len(seq1) for _ in range(len(seq2))]

    max_in_matrix = [0,(0,0)]
    arrows = {}
    
    for j in range(1, len(seq2)):
        for i in range(1, len(seq1)):
            mismatch = H[j-1][i-1] + S(seq1[i], seq2[j])
            delete = H[j][i-1] - W_1
            insert = H[j-1][i] - W_1
            value = max(0, mismatch, delete, insert)
                    
            H[j][i] = value
            
            if value == 0:
                continue
                
            arrow = []
            if value == mismatch:
                arrow.append((i-1, j-1))
            if value == delete:
                arrow.append((i-1, j))
            if value == insert:
                arrow.append((i, j-1))
            arrows[(i, j)] = arrow
                
            if value >= max_in_matrix[0]:
                max_in_matrix = [value, (i, j)]

    return (H, max_in_matrix, arrows)

In [6]:
def backtrack(H: list[list[int]], pos: tuple, arrows: dict) -> list[tuple]:
    i, j = pos[1]
    path = []
    
    while H[j][i]:
        path.append((i, j))
        i, j = arrows[(i,j)][0]
    return path[::-1]

In [7]:
def get_optimal_allignment(path: dict, seq1: str, seq2: str) -> tuple:
    seq1_allign, seq2_allign = '', ''
    for k, step in enumerate(path):
        i, j = step
        seq1_allign += seq1[i] if path[k-1][0] != path[k][0] else '-'
        seq2_allign += seq2[j] if path[k-1][1] != path[k][1] else '-'
    return seq1_allign, seq2_allign

In [32]:
def smith_waterman(seq1: str, seq2: str, print_allign: bool = False, print_matrix: bool = False) -> tuple:
    seq1, seq2 = '_' + seq1, '_' + seq2
    H, max_in_matrix, arrows = scoring_matrix(seq1, seq2) 
    path = backtrack(H, max_in_matrix, arrows)
    alligned = get_optimal_allignment(path, seq1, seq2)
    
    if print_matrix: print_scoring_matrix(H, seq1, seq2) 
    if print_allign: print_optimal_allignment(*alligned)
    
    return alligned

In [42]:
smith_waterman('TGTTACGG', 'GGTTGACTA', print_allign=True, print_matrix=True)

    _   T   G   T   T   A   C   G   G   
_   0   0   0   0   0   0   0   0   0   
G   0   0   3   1   0   0   0   3   3   
G   0   0   3   1   0   0   0   3   6   
T   0   3   1   6   4   2   0   1   4   
T   0   3   1   4   9   7   5   3   2   
G   0   1   6   4   7   6   4   8   6   
A   0   0   4   3   5   10  8   6   5   
C   0   0   2   1   3   8   13  11  9   
T   0   3   1   5   4   6   11  10  8   
A   0   1   0   3   2   7   9   8   7   
GTT-AC
||| ||
GTTGAC


('GTT-AC', 'GTTGAC')

In [40]:
smith_waterman(random_sequence(100), random_sequence(100))

('GATAAATGTGTGGTGT-GTAAAAGGCGTTC-TGTGC-A---TACC---CC-ATAGC-C-T--ACT-CCTATGGT-TC-CTAT-TGATGAGAGATCCCCGTTAT-G-G',
 'GAT--ATTTGT-GTATCGT---TGG-ATTCAT-T-CTACCCTACCAATCCAATCGCTCGTAAAGTACCGGTGGTACCGCT-TCT-AT-AG-GA-AACTGTTATAGAG')